# Machine learning project 

## Predicting the disease type using X ray images 

### Built of Cnn model after all preprocessing has been done

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import cv2
%matplotlib inline

In [2]:
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [3]:
x_train_norm=np.load('normalized_image_array.npy') #loading the array.

In [4]:
print(len(x_train_norm))
print(x_train_norm.shape)

18577
(18577, 128, 128, 3)


Our final training data is an 18577x128x128x3 array. This mean that it is an array of 18577 images of size 128x128 with 3 as the RGB spectrum.

In [5]:
from sklearn.cross_validation import train_test_split
# Split the 'features' and 'income' data into training and testing sets
target=train_data['detected']
X_train, X_test, y_train, y_test = train_test_split(x_train_norm, target, test_size = 0.3, random_state = 42)

/home/adityajainkld/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**Cross validation** Splitting our data into training and testing part

In [6]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

/home/adityajainkld/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Label encoding is a process by which categorical variables are converted into a form that could be provided to trainning algorithms to do a better job in prediction.

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_y_train)

In [8]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_y_test)

In [9]:
y_train[3]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [10]:
num_classes=y_train.shape[1]

### Building the model and training it


We will use keras module for this problem. Keras is a high-level API written in Python. It runs on top of Tensorflow, CNTK or Theano. It is relatively easy to use and is very fast.

In [11]:
# Simple CNN model
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import Dropout
from keras.layers import Flatten,Convolution2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras

## Model #1

In [35]:
model = Sequential()

'''
First set of three layers
Image Size: 128 x 128
nb_filters = 64
kernel_size = 4,4
'''

nb_filters = 32
kernel_size = (2,2)
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),
                 padding='valid',
                 strides=1,
                 input_shape = (128,128,3)))
model.add(Activation('relu'))

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
'''
Second set of three layers
Image Size: 128 x 128
nb_filters = 62
kernel_size = 4,4
'''

nb_filters = 64
kernel_size = (4, 4)

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

# model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
# model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
'''
Third set of three layers
Image Size: 64 x 64
nb_filters = 128
kernel_size = 8,8
'''

nb_filters = 128
kernel_size = (8, 8)

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

# model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
# model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(12, 12)))
model.add(Dropout(0.5))

model.add(Flatten())
print("Model flattened out to: ", model.output_shape)

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())


Model flattened out to:  (None, 128)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 127, 127, 32)      416       
_________________________________________________________________
activation_1 (Activation)    (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      4128      
_________________________________________________________________
activation_2 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 125, 125, 32)      4128      
_________________________________________________________________
activation_3 (Activation)    (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 62,

In [37]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)

In [18]:
from keras.callbacks import ModelCheckpoint
filepath="ml_project_cnn_wts.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
early_stops = EarlyStopping(patience=7, monitor='acc')
callbacks_list = [checkpoint,early_stops]

In [ ]:
hist_model=model.fit(X_train,y_train,epochs=7,batch_size=128,callbacks=callbacks_list,verbose=1)

Epoch 1/7
13003/13003 [==============================] - 1316s 101ms/step - loss: 2.1912 - acc: 0.3122

Epoch 00001: acc did not improve
Epoch 2/7
13003/13003 [==============================] - 1324s 102ms/step - loss: 2.1902 - acc: 0.3122

Epoch 00002: acc did not improve
Epoch 3/7
13003/13003 [==============================] - 1312s 101ms/step - loss: 2.1891 - acc: 0.3122

Epoch 00003: acc did not improve
Epoch 4/7
13003/13003 [==============================] - 1318s 101ms/step - loss: 2.1892 - acc: 0.3122

Epoch 00004: acc did not improve
Epoch 5/7
 7424/13003 [================>.............] - ETA: 9:22 - loss: 2.1830 - acc: 0.3168

In [15]:
model.load_weights('ml_project_cnn_wts.best.hdf5')

In [43]:
from keras.models import model_from_json
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("ml_project_cnn_wts.best.hdf5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)

print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 29.92%


## MODEL #2

In [44]:
# Create the model

model = Sequential()
model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 30, 30, 64)        0         
__________

In [13]:
early_stops = EarlyStopping(patience=7, monitor='acc')

In [14]:
from keras.callbacks import ModelCheckpoint
filepath="ml_project_cnn_wts_2.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
early_stops = EarlyStopping(patience=7, monitor='acc')
callbacks_list = [checkpoint,early_stops]

In [17]:
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64,verbose=3)
model1_adam=model.fit(X_train, y_train, batch_size=128, epochs=25,verbose=1)

# Final evaluation of the model
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/25
13003/13003 [==============================] - 891s 69ms/step - loss: 2.2068 - acc: 0.3053
Epoch 2/25
13003/13003 [==============================] - 882s 68ms/step - loss: 2.1311 - acc: 0.3139
Epoch 3/25
13003/13003 [==============================] - 886s 68ms/step - loss: 2.1016 - acc: 0.3198
Epoch 4/25
13003/13003 [==============================] - 888s 68ms/step - loss: 2.0788 - acc: 0.3278
Epoch 5/25
13003/13003 [==============================] - 888s 68ms/step - loss: 2.0536 - acc: 0.3335
Epoch 6/25
13003/13003 [==============================] - 889s 68ms/step - loss: 2.0329 - acc: 0.3426
Epoch 7/25
13003/13003 [==============================] - 884s 68ms/step - loss: 2.0041 - acc: 0.3492
Epoch 8/25
13003/13003 [==============================] - 883s 68ms/step - loss: 1.9748 - acc: 0.3560
Epoch 9/25
13003/13003 [==============================] - 890s 68ms/step - loss: 1.9474 - acc: 0.3619
Epoch 10/25
13003/13003 [==============================] - 888s 68ms/step - loss: 

In [18]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 30.27%


## MODEL #3

Deep networks need large amount of training data to achieve good performance. To build a powerful image classifier using very little training data, **image augmentation** is usually required to boost the performance of deep networks. Image augmentation artificially creates training images through different ways of processing or combination of multiple processing, such as **random rotation, shifts, shear and flips, etc.**